In [8]:
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

In [5]:
dtw_test1 = np.load("dtw_test1.npy")
dtw_test2 = np.load("dtw_test2.npy")
dtw_train1 = np.load("dtw_array1.npy")
dtw_train2 = np.load("dtw_array2.npy")
test_y = np.load("test_y.npy")

In [6]:
train_label = []
for i in range(1,13):
    train_label.extend([i,i,i,i,i])

In [40]:
from sklearn import neighbors
neigh = neighbors.KNeighborsClassifier(3)
neigh.fit(dtw_train2, train_label) 
pred2 = neigh.predict(dtw_test2)
print(accuracy_score(test_y,pred2))

neigh = neighbors.KNeighborsClassifier(3)
neigh.fit(dtw_train1, train_label) 
pred1 = neigh.predict(dtw_test1)
print(accuracy_score(test_y,pred1))

neigh = RandomForestClassifier( n_estimators=1000)
neigh.fit(dtw_train1, train_label) 
pred3 = neigh.predict(dtw_test1)
print(accuracy_score(test_y,pred3))

neigh = RandomForestClassifier( n_estimators=1000)
neigh.fit(dtw_train2, train_label) 
pred4 = neigh.predict(dtw_test2)
print(accuracy_score(test_y,pred4))

concat = np.empty([60,120])
concat[:, 0:60] = dtw_train1
concat[:, 60:] = dtw_train2
test = np.empty([488,120])
test[:, 0:60] = dtw_test1
test[:, 60:] = dtw_test2

neigh = RandomForestClassifier( n_estimators=100)
neigh.fit(concat, train_label) 
pred5 = neigh.predict(test)
print(accuracy_score(test_y,pred5))

0.180327868852
0.334016393443
0.311475409836
0.170081967213
0.303278688525


In [25]:
#creating the semisupervized batch
solution2 = []
for i, predicted in enumerate(pred2):
    if predicted == test_y[i]:
        solution2.append(i)
solution1 = []
for i, predicted in enumerate(pred1):
    if predicted == test_y[i]:
        solution1.append(i)
difference = list(set(solution2)-set(solution1))
print(len(solution1),len(solution2),len(solution2)-len(difference))

(157, 163, 128)


In [41]:
out = open("submission.txt","w")

test_index = np.load("test_index.npy")
numb = 0
for index,y in enumerate(test_y):
    result = 6
    if y == pred5[index]  :
        result = 1
        numb = numb + 1
    else:
        result = 0
    out.write(str(test_index[index])+",")
    out.write(str(result))
    out.write("\n")
out.close()
print(float(numb)/488)

0.303278688525


In [24]:
numb = 0
for index,y in enumerate(test_y):
    result = 6
    if y == pred1[index] or y == pred2[index]:
        numb = numb + 1
print(numb)

238


In [25]:
#semi supervized szettek elkészítése
int_train = 0
int_test = 0
semi_train1 = np.empty([numb,60])
semi_train2 = np.empty([numb,60])
semi_label = np.empty([numb])
for index,y in enumerate(test_y):
    if y == pred1[index] or y == pred2[index]:
        semi_train1[int_train, :] = dtw_test1[index, :]
        semi_train2[int_train, :] = dtw_test2[index, :]
        semi_label[int_train] = y
        int_train = int_train + 1

In [26]:
con_semi_train1 = np.empty([numb+60,60])
con_semi_train1[0:60,:] = dtw_train1[:,:]
con_semi_train1[60:,:] = semi_train1[:, :]
con_semi_train2 = np.empty([numb+60,60])
con_semi_train2[0:60,:] = dtw_train2[:,:]
con_semi_train2[60:,:] = semi_train2[:, :]
con_semi_label = np.empty([numb+60])
con_semi_label[0:60] = train_label[:]
con_semi_label[60:] = semi_label[:]

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf = neighbors.KNeighborsClassifier(1)
clf = clf.fit(con_semi_train1, con_semi_label)
pred1 = clf.predict(dtw_test1)
accuracy_score(pred1, test_y)

0.48770491803278687

In [28]:
clf = neighbors.KNeighborsClassifier(1)
clf = clf.fit(con_semi_train2, con_semi_label)
pred2 = clf.predict(dtw_test2)
accuracy_score(pred2, test_y)

0.48770491803278687

In [37]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV()
clf = clf.fit(con_semi_train2, con_semi_label)
pred2 = clf.predict(dtw_test2)
accuracy_score(pred2, test_y)

0.49795081967213117